## Урок 7. Дектирование объектов

### Практическое задание

**1. Сделайте краткий обзор какой-нибудь научной работы посвященной тому или иному алгоритму для object detection, который не рассматривался на уроке. Проведите анализ: Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур? В чем плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении данной архитектуры на практике?**

**EfficientDet** - масштабируемое и эффективное обнаружение объектов  
источники:  
https://github.com/google/automl/tree/master/efficientdet  
https://arxiv.org/pdf/1911.09070.pdf  
https://habr.com/ru/post/503766/

Общая архитектура EfficientDet в значительной степени соответствует парадигме одноступенчатых (one-stage) детекторов. За основу взята EfficientNet(класс моделей, который получился из изучения масштабирования scaling моделей и балансирования между собой глубины и ширины (количества каналов) сети, а также разрешения изображений в сети), предобученная на ImageNet, к ней приделан слой со взвешенной двунаправленной пирамидой признаков (BiFPN - позволяет повысить эффективность модели за счет удаления потенциально "лишних" узлов и добавления дополнительных ребер из исходного в выходной узел, чтобы объединить больше функций без дополнительных затрат), за которым идёт классовая и блочная сеть для генерации предсказаний класса объекта и ограничивающего прямоугольника соответственно.

![](https://raw.githubusercontent.com/google/automl/master/efficientdet/g3doc/network.png)

Также в EfficientDet используется нормализованное relu (Relu(w[i])/(sum(Relu(w[k])) + 0.001)
где sum Суммирует по всем k) вместо SoftMax, в основе которой лежит метод быстрой нормализации слияния, который обеспечивает такую же точность, что и слияние на основе Softmax, но работает быстрее в 1,26-1,31 раза на графических процессорах.

Помимо этого, была улучшена производительность в классификации изображений с помощью совместного увеличения всех размеров сети, глубины и входного разрешения.

Благодаря всему этому, новое семейство детекторов под названием EfficientDet, последовательно достигает большей точности и эффективности, чем предшествующий уровень техники, при широком спектре ограничений по ресурсам. Так, например, по заявлению разработчиков, EfficientDet-D7 достигает современной точности с меньшим количеством параметров и FLOPS, чем лучший из существующих детекторов. EfficientDet также в 3,2 раза быстрее на GPU и в 8,1 раза быстрее на CPU.

![](https://github.com/google/automl/raw/master/efficientdet/g3doc/flops.png)

Но это все в теории. На практике все оказывается не так радужно.

Так, например, авторы EfficientDet уточняют, что FPS-скорость для Yolov3 они взяли из yolo-статьи, где её измеряли на TitanX/M40 maxwell, а FPS-скорость для EfficientDet они измеряли на TitanV volta, которая на 2 поколения новее и в несколько раз быстрее. Только поэтому в их отчете EfficientDet получилась чуть лучше по скорости и точности, чем старая Yolov3.

![](https://habrastorage.org/webt/vp/3p/zd/vp3pzd9kl84cs-pvdok1fp_b8mq.png)

По сути, авторы EfficientDet разработали эффективную с точки зрения формальных параметров BFLOPS, но не эффективную с точки зрения производительности, поэтому в последствии, Google отказались от большинства фич, которые радикально уменьшают BFLOPS  
https://ai.googleblog.com/2019/08/efficientnet-edgetpu-creating.html

Итого: возможно в каких-то специализированных задачах, для которых она изначально и задумывалась авторами, EfficientDet и будет иметь преимущества, но на данный момент существуют более универсальные и оптимальные с точки зрения быстродейтствия на GPU/NPU/VPU/TPU конкуренты (например YOLOv4)

Пример визуализации работы EfficientDet-D0

![](https://github.com/google/automl/raw/master/efficientdet/g3doc/street.jpg)